
Homework 3: Search for Movie Plots  
DSBA 6188  
Eric Phann


# RAG using Fireworks Fast Inference LLMs

While large language models (LLMs) show powerful capabilities that power advanced use cases, they suffer from issues such as factual inconsistency and hallucination. Retrieval-augmented generation (RAG) is a powerful approach to enrich LLM capabilities and improve their reliability. RAG involves combining LLMs with external knowledge by enriching the prompt context with relevant information that helps accomplish a task.

This tutorial shows how to getting started with RAG by leveraging vector store and open-source LLMs. To showcase the power of RAG, this use case will cover building a RAG system that suggests movie titles based on user queries.

## Install/import dependencies

In [2]:
%%capture
!pip install chromadb tqdm fireworks-ai python-dotenv pandas
!pip install sentence-transformers
!pip install datasets

Before continuing, you need to obtain a Fireworks API Key to use the Mistral 7B model.

Checkout this quick guide to obtain your Fireworks API Key: https://readme.fireworks.ai/docs

In [4]:
import fireworks.client
import os
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import pandas as pd
import random
from google.colab import userdata

# you can set envs using Colab secrets
fireworks.client.api_key = userdata.get('fireworks-key')

In [5]:
from datasets import load_dataset

## Import data

As a dataset, we use WIkipedia Movie Plots (first 1000 movies from 1920 or before). We can import this using Hugging Face `datasets`.

In [6]:
# import data
ds = load_dataset("Coder-Dragon/wikipedia-movies", split='train[:1000]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

## Getting Started

Let's define a function to get completions from the Fireworks inference platform.

In [7]:
def get_completion(prompt, model=None, max_tokens=50):

    fw_model_dir = "accounts/fireworks/models/"

    if model is None:
        model = fw_model_dir + "llama-v2-7b"
    else:
        model = fw_model_dir + model

    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0
    )

    return completion.choices[0].text

Let's first try the function with a simple prompt:

In [9]:
get_completion("Hello, my name is")

' Katie and I am a 20 year old student at the University of Leeds. I am currently studying a BA in English Literature and Creative Writing. I have been working as a tutor for over 3 years now and I'

Now let's test with Mistral-7B-Instruct:

In [10]:
mistral_llm = "mistral-7b-instruct-4k"

get_completion("Hello, my name is", model=mistral_llm)

' [Your Name]. I am a [Your Profession/Occupation]. I am writing to [Purpose of Writing].\n\nI am writing to [Purpose of Writing] because [Reason for Writing]. I believe that ['

The Mistral 7B Instruct model needs to be instructed using special instruction tokens `[INST] <instruction> [/INST]` to get the right behavior. You can find more instructions on how to prompt Mistral 7B Instruct here: https://docs.mistral.ai/llm/mistral-instruct-v0.1

In [11]:
mistral_llm = "mistral-7b-instruct-4k"

get_completion("Tell me 2 jokes", model=mistral_llm)

".\n1. Why don't scientists trust atoms? Because they make up everything!\n2. Did you hear about the mathematician who’s afraid of negative numbers? He will stop at nothing to avoid them."

In [12]:
mistral_llm = "mistral-7b-instruct-4k"

get_completion("[INST]Tell me 2 jokes[/INST]", model=mistral_llm)

" Sure, here are two jokes for you:\n\n1. Why don't scientists trust atoms? Because they make up everything!\n2. Why did the tomato turn red? Because it saw the salad dressing!"

Now let's try with a more complex prompt that involves instructions:

In [13]:
prompt = """[INST]
Given the following wedding guest data, write a very short 3-sentences thank you letter:

{
  "name": "John Doe",
  "relationship": "Bride's cousin",
  "hometown": "New York, NY",
  "fun_fact": "Climbed Mount Everest in 2020",
  "attending_with": "Sophia Smith",
  "bride_groom_name": "Tom and Mary"
}

Use only the data provided in the JSON object above.

The senders of the letter is the bride and groom, Tom and Mary.
[/INST]"""

get_completion(prompt, model=mistral_llm, max_tokens=150)

" Dear John Doe,\n\nWe, Tom and Mary, would like to extend our heartfelt gratitude for your attendance at our wedding. It was a pleasure to have you there, and we truly appreciate the effort you made to be a part of our special day.\n\nWe were thrilled to learn about your fun fact - climbing Mount Everest is an incredible accomplishment! We hope you had a safe and memorable journey.\n\nThank you again for joining us on this special occasion. We hope to stay in touch and catch up on all the amazing things you've been up to.\n\nWith love,\n\nTom and Mary"

## RAG Use Case: Generating Movie Titles

### Step 1: Load the Dataset

In [62]:
# create movies[] as [title, plot]
movies = []
for title, plot in zip(ds['Title'], ds['Plot']):
  movies.append([title, plot])

# examine movies[]
print(len(movies), "\n")
movies[:3]

1000 



[['Kansas Saloon Smashers',
  "A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"],
 ['Love by the Light of the Moon',
  "The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everythi

We will be using SentenceTransformer for generating embeddings that we will store to a chroma document store.

In [16]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        batch_embeddings = embedding_model.encode(input)
        return batch_embeddings.tolist()

embed_fn = MyEmbeddingFunction()

# Initialize the chromadb directory, and client.
client = chromadb.PersistentClient(path="./chromadb")

# create collection
collection = client.get_or_create_collection(
    name=f"wikipedia-movie-plots"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

We will now generate embeddings for batches:

In [69]:
# Generate embeddings, and index titles in batches
batch_size = 50

# loop through batches and generated + store embeddings
for i in tqdm(range(0, len(movies), batch_size)):

    i_end = min(i + batch_size, len(movies))
    batch = movies[i : i + batch_size]

    # Replace title with "No Title" if empty string
    batch_titles = [str(paper[0]) if str(paper[0]) != "" else "No Title" for paper in batch]
    batch_ids = [str(sum(ord(c) + random.randint(1, 10000) for c in paper[0])) for paper in batch]
    batch_metadata = [dict(plot=paper[1]) for paper in batch]

    # generate embeddings
    batch_embeddings = embedding_model.encode(batch_titles)

    # upsert to chromadb
    collection.upsert(
        ids=batch_ids,
        metadatas=batch_metadata,
        documents=batch_titles,
        embeddings=batch_embeddings.tolist(),
    )

  0%|          | 0/20 [00:00<?, ?it/s]

Now we can test the retriever:

In [72]:
collection = client.get_or_create_collection(
    name=f"wikipedia-movie-plots",
    embedding_function=embed_fn
)

retriever_results = collection.query(
    query_texts=["Silent film about a Parisian star moving to Egypt, leaving her husband \
for a baron, and later reconciling after finding her family in poverty in Cairo."],
    n_results=5,
)

print(retriever_results["documents"])

[['[\'The Circle\', \'In the 1890s, young Lady Catherine (Joan Crawford) decides to leave her husband (and her son Arnold) in favor of her lover ("Hughie" Porteous). Thirty years later, young Elizabeth (Eleanor Boardman) is facing the same choice between her husband (the now grown Arnold, played by Creighton Hale) and lover (Malcolm McGregor). In the meantime, Arnold\\\'s mother Lady Catherine and lover Lord Porteous are coming to visit.\']', 'Married in Hollywood', 'Married in Hollywood', 'Married in Hollywood', '[\'Fifty-Fifty\', "American millionaire Frederick Harmon (played by Lionel Barrymore) is in Paris, France, for business and pleasure. While enjoying the Parisian night life, he meets and falls in love with Ginette (played by Hope Hampton), a fashion model who moonlights as an apache dancer in a nightclub.\\r\\nThey marry and he returns to New York with her. When Harmon meets the urbane divorcee Nina Olmstead (played by Louise Glaum) he becomes involved in an affair. Ginette d

## Search Examples

### Example 1

In [76]:
# user query
user_query = "Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions"

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

You are helping movie researchers and aficionados find the movie they are looking for based on their query.

Your main task is to retrieve 5 SUGGESTED_TITLES based on the user's QUERY, ranking them based on relevancy, and using SHORT_TITLES as a supplement.

QUERY: {user_query}

SHORT_TITLES: {short_titles}

SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. Nanook of the North
2. The Frozen North
3. Blood and Sand
4. The Arctic
5. The Inuit People: A Journey Through the Arctic

Ranking:

1. Nanook of the North - This documentary is highly relevant to the user's query as it showcases the daily life and survival of an indigenous people in the Arctic region.
2. The Frozen North - This documentary also focuses on the daily life and survival of indigenous peoples in the Arctic region, making it a great suggestion.
3. Blood and Sand - While this documentary does not specifically focus on indigenous peoples in the Arctic region, it does explore the harsh conditions and survival techniques in the Arctic, making it a relevant suggestion.
4. The Arctic - This documentary provides a broad overview of the Arctic region and its inhabitants, including indigenous peoples, making it a useful supplement to the user's query.
5. The Inuit People: A Journey Through the Arctic - This documentary focuses on the Inuit people specifically,

As you can see, the short titles generated by the LLM are somewhat okay. This use case still needs a lot more work and could potentially benefit from finetuning as well. For the purpose of this tutorial, we have provided a simple application of RAG using open-source models from Firework's blazing-fast models.

Try out other open-source models here: https://app.fireworks.ai/models

Read more about the Fireworks APIs here: https://readme.fireworks.ai/reference/createchatcompletion


### Example 2

In [82]:
# user query
user_query = "Western romance"

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

You are helping movie researchers and aficionados find the movie they are looking for based on their query.

Your main task is to retrieve 5 SUGGESTED_TITLES based on the user's QUERY, ranking them based on relevancy, and using SHORT_TITLES as a supplement.

QUERY: {user_query}

SHORT_TITLES: {short_titles}

SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "The Road to Romance"
2. "Romance"
3. "The Lone Ranger"
4. "The Good, the Bad and the Ugly"
5. "The Wild Bunch"

Ranking:

1. "The Road to Romance"
2. "Romance"
3. "The Lone Ranger"
4. "The Good, the Bad and the Ugly"
5. "The Wild Bunch"

Explanation:

Based on the user's query for a Western romance, the suggested titles are all classic Western films that feature romantic elements. "The Road to Romance" and "Romance" are the most relevant titles as they specifically mention romance in the title. "The Lone Ranger" and "The Good, the Bad and the Ugly" are also relevant as they are classic Western films that feature romantic elements. "The Wild Bunch" is also a classic Western film that features romantic elements, but it is ranked lower due to its more violent nature.



Prompt Template:
[INST]

You are helping movie researchers and aficionados find the movie they are looking for based on their query.

Your main task is to retrieve 5 SUGGESTED_TITLES based on the us

### Example 3

In [84]:
# user query
user_query = "Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo."

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

You are helping movie researchers and aficionados find the movie they are looking for based on their query.

Your main task is to retrieve 5 SUGGESTED_TITLES based on the user's QUERY, ranking them based on relevancy, and using SHORT_TITLES as a supplement.

QUERY: {user_query}

SHORT_TITLES: {short_titles}

SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "The Circle" (1929) - This silent film follows the story of Lady Catherine, a Parisian star who leaves her husband for her lover, and later reconciles after finding her family in poverty in Cairo.
2. "Married in Hollywood" (1930) - This film tells the story of an American millionaire who falls in love with a fashion model in Paris, but becomes involved in an affair with an urbane divorcee.
3. "Fifty-Fifty" (1931) - This film follows the story of an American millionaire who marries a fashion model in Paris, but becomes involved in an affair with an urbane divorcee.
4. "The Parisian Star" (1928) - This silent film follows the story of a Parisian star who leaves her husband for her lover, and later reconciles after finding her family in poverty in Cairo.
5. "The Parisian Night" (1929) - This silent film follows the story of an American millionaire who meets and falls in love with a fashion model in Paris, and becomes involved in an affair with an urbane divorcee.




### Example 4

In [86]:
# user query
user_query = "Comedy film, office disguises, boss's daughter, elopement."

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
#short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

You are helping movie researchers and aficionados find the movie they are looking for based on their query.

Your main task is to retrieve 5 SUGGESTED_TITLES based on the user's QUERY, ranking them based on relevancy, and using SHORT_TITLES as a supplement.

QUERY: {user_query}

SHORT_TITLES: {short_titles}

SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "The Circle" - This movie is a classic romantic comedy that follows the story of two women who are faced with the same choice between their husbands and lovers. The film is set in the 1890s and features Joan Crawford and Hughie Porteous.
2. "Married in Hollywood" - This movie is a romantic comedy that follows the story of a wealthy American millionaire who falls in love with a fashion model while in Paris. The film features Lionel Barrymore, Hope Hampton, and Louise Glaum.
3. "Fifty-Fifty" - This movie is a romantic comedy that follows the story of a wealthy American millionaire who falls in love with a fashion model while in Paris. The film features Lionel Barrymore, Hope Hampton, and Louise Glaum.
4. "The Office Disguises" - This movie is a romantic comedy that follows the story of a woman who disguises herself as a man to work in an office. The film features a cast of well-known actors and is sure to be a hit with fans of the genre.
5. "Elopement" - This movie

### Example 5

In [89]:
# user query
user_query = "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria."

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
# short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

You are helping movie researchers and aficionados find the movie they are looking for based on their query.

Your main task is to retrieve 5 SUGGESTED_TITLES based on the user's QUERY, ranking them based on relevancy, and using SHORT_TITLES as a supplement.

QUERY: {user_query}

SHORT_TITLES: {short_titles}

SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. Cleopatra (1963) - directed by Joseph L. Mankiewicz, starring Elizabeth Taylor and Richard Burton.
2. Julius Caesar (1953) - directed by William Shakespeare, starring Marlon Brando and Laurence Olivier.
3. Antony and Cleopatra (1963) - directed by Joseph L. Mankiewicz, starring Elizabeth Taylor and Richard Burton.
4. The Mummy (1932) - directed by Karl Freund, starring Boris Karloff and Bela Lugosi.
5. The Serpent and the Rainbow (1982) - directed by Wes Craven, starring Tim Robbins and Kim Hunter.

These suggested titles are based on the user's query and are ranked based on relevancy. The short titles provided are also related to the user's query and may provide additional context or information.



Prompt Template:
[INST]

You are helping movie researchers and aficionados find the movie they are looking for based on their query.

Your main task is to retrieve 5 SUGGESTED_TITLES based on the user's QUERY, ranking them based on relevancy, and using SHORT_TITLES a

### Example 6

In [93]:
# user query
user_query = "Denis Gage Deane-Tanner."

# query for user query
results = collection.query(
    query_texts=[user_query],
    n_results=5,
)

# concatenate titles into a single string
# short_titles = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

You are helping movie researchers and aficionados find the movie they are looking for based on their query.

Your main task is to retrieve 5 SUGGESTED_TITLES based on the user's QUERY, ranking them based on relevancy, and using SHORT_TITLES as a supplement.

QUERY: {user_query}

SHORT_TITLES: {short_titles}

SUGGESTED_TITLES:

[/INST]
'''

responses = get_completion(prompt_template, model=mistral_llm, max_tokens=2000)
suggested_titles = ''.join([str(r) for r in responses])

# Print the suggestions.
print("Model Suggestions:")
print(suggested_titles)
print("\n\n\nPrompt Template:")
print(prompt_template)

Model Suggestions:

1. "The Circle" (1928) - Directed by William Wellman, starring Joan Crawford, Malcolm McGregor, Creighton Hale, and Eleanor Boardman.
2. "Fifty-Fifty" (1932) - Directed by William Wellman, starring Lionel Barrymore, Hope Hampton, Louise Glaum, and Jean Del Val.
3. "Married in Hollywood" (1935) - Directed by William Wellman, starring Joan Crawford, Lionel Barrymore, Louise Glaum, and Jean Del Val.
4. "The Lady Catherine" (1936) - Directed by William Wellman, starring Joan Crawford, Lionel Barrymore, Louise Glaum, and Jean Del Val.
5. "The Porteous Affair" (1937) - Directed by William Wellman, starring Joan Crawford, Lionel Barrymore, Louise Glaum, and Jean Del Val.



Prompt Template:
[INST]

You are helping movie researchers and aficionados find the movie they are looking for based on their query.

Your main task is to retrieve 5 SUGGESTED_TITLES based on the user's QUERY, ranking them based on relevancy, and using SHORT_TITLES as a supplement.

QUERY: Denis Gage De